In this jupyter notebook, we will be analyzing stochastic simulations of 1x/2x naive, and 1x/2x differentiation and terminal differentiation as shown in Figure 3, and Figures Supplementary Figures 17-26. In these simulations, we are characterizing the perfomance of each of these architectures and how they are impacted by burden level, differentiation rate (for differentiation architectures), and plasmid effects: both plasmid mutation, and plasmid loss cobined with communal antibiotic resistance.

In [1]:
# NumPy and odeint, our workhorses
import numpy as np
import scipy.integrate
import scipy.signal
import numba 
import pandas as pd
import time
# Plotting modules
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

# import bioscrape as bs
# import bebi103
# import corner
# import emcee
import numba
import time
import pickle
from cycler import cycler
import itertools
import multiprocessing as MP
import warnings
# from differentiation_functions_ODE_integrase_mutations_abx_cheating_additive_burden_20211116 import *

sns.set_context("talk", font_scale=1.5, rc={"lines.linewidth": 1.5})
sns.set_style("ticks")
sns.set_style({"xtick.direction": "in","ytick.direction": "in"})

#%config InlineBackend.figure_f.ormats=['svg']

mpl.rc('axes', prop_cycle=(cycler('color', ['r', 'k', 'b','g','y','m','c']) ))

mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42

#mpl.rc('text', usetex=False)
#mpl.rc('text.latex', preamble=r'\usepackage{helvet}
#\renewcommand\familydefault{\sfdefault}\usepackage{sansmath}\sansmath')
# mpl.rc('font',**{'family':'sans-serif','sans-serif':['Helvetica Neue'], 
#                  'serif': ['Helvetica Neue']})

mpl.rc('font',**{'family':'sans-serif','sans-serif':['Helvetica'], 
                 'serif': ['Helvetica']})
tw = 1.5
sns.set_style({"xtick.major.size": 3, "ytick.major.size": 3,
               "xtick.minor.size": 2, "ytick.minor.size": 2,
               'axes.labelsize': 16, 'axes.titlesize': 16,
               'xtick.major.width': tw, 'xtick.minor.width': tw,
               'ytick.major.width': tw, 'ytick.minor.width': tw})

mpl.rc('xtick', labelsize=14) 
mpl.rc('ytick', labelsize=14)
mpl.rc('axes', linewidth=1.5)
mpl.rc('legend', fontsize=14)
mpl.rc('legend', frameon=False)
mpl.rc('figure', figsize=(8.5,15))
%matplotlib qt

The simulations were run on the Caltech HPC, and the results saved as .pkl files. The .pkl files are quite large, probably mostly because we are storing the endpoint cell count for all genotypes and not just the categories. First will write a function that will read in .pkl files, get summary endpoint counts for:

    * Naive: producers, mutants; degraders and non-degraders/cells that have lost the plasmid (for antibiotic degradation simulations); and those that have have mutated the plasmid (for plasmid mutation simulations). For convenience, degraders is used in the analysis below to refer to both cells that have retained the plasmid in antibiotic degradation simulations, and for cells that have not mutated the plasmid in simulations incorporating plasmid mutations.
    
    * Differentiation: producers, progenitors, mutants (including non-producers/non-differentiators), degraders, cheaters. 

Each of these will be separate columns, additionally I will add a column which contains the 'winning' genotype from the endpoint of the final growth of the simulation

The code below is to read in individual .pkl files from simulations. Skip below to read in larger .pkl files to continue with anaylsis.

In [2]:
import glob
fnames = glob.glob('../20211129_hpc_simulations/diff_stoch/*.pkl')

In [3]:
def naive_pkl_aggregation(fnames):
    agg_summary_columns = np.array(['circuit','selection','muP_single_cassette','stochastic','n_cassettes',
                                 'muN','muP','burden','Kpop','kMB','kPL','Vmax','Km','MIC','abx','V','rep',
                                 'prod_frac_median','prod_frac_avg','total_production','production_rate',
                                 'producers','mutants','degraders','cheaters','pop_tot','winning_genotype','winning_gen_endpoint',
                                 'production_array','time_array'])
    agg_summary_list = []
    for fname in fnames:
        df = pd.read_pickle(fname)
        for i in range(len(df)):
            results = df.results.values[i]
            ind_dict = df.pn_ind_dict.values[i]
            add_dict = {}
            add_list = []
            add_dict['burden'] = int(round(100*(df.muN.values[i]-df.muP.values[i])/df.muN.values[i],2))
            for key in list(ind_dict.keys()):
                add_dict[key] = results[:,ind_dict[key]].sum(axis=1)
            add_dict['pop_tot'] = results[:,0:-2].sum(axis=1)
            genotypes = df.genotypes.values[i]
            add_dict['winning_genotype'] = genotypes[results[-1][0:-2].argmax()]
            add_dict['winning_gen_endpoint'] = results[-1][0:-2].max()
            for col in agg_summary_columns:
                if col in df.columns:
                    add_list.append(df[col].values[i])
                elif col in add_dict.keys():
                    add_list.append(add_dict[col])
                else:
                    print(f'{col} was not found')
            agg_summary_list.append(add_list)
    return pd.DataFrame(columns=agg_summary_columns,data=agg_summary_list)
            
                    
            

In [4]:
def diff_pkl_aggregation(fnames):
    agg_summary_columns = np.array(['circuit','stochastic','n_cassettes','n_int_cassettes','selection','CIIE','n_div',
                                 'muN','muP','burden','Kpop','kdiff','kMB','kMD','kMI','kPL','Vmax','Km',
                                 'MIC','abx','V','rep','prod_frac_median','prod_frac_avg','total_production',
                                  'production_rate','producers', 'progenitors', 'mutants', 'degraders', 
                                  'cheaters','pop_tot','winning_genotype','winning_gen_endpoint','production_array','time_array'])
    agg_summary_list = []
    for fname in fnames:
        df = pd.read_pickle(fname)
        for i in range(len(df)):
            results = df.results.values[i]
            ind_dict = df.species_ind_dict.values[i]
            add_dict = {}
            add_list = []
            add_dict['burden'] = int(round(100*(df.muN.values[i]-df.muP.values[i])/df.muN.values[i],2))
            for key in list(ind_dict.keys()):
                add_dict[key] = results[:,ind_dict[key]].sum(axis=1)
            add_dict['pop_tot'] = results[:,0:-2].sum(axis=1)
            genotypes = df.genotypes.values[i]
            add_dict['winning_genotype'] = genotypes[results[-1][0:-2].argmax()]
            add_dict['winning_gen_endpoint'] = results[-1][0:-2].max()
            for col in agg_summary_columns:
                if col in df.columns:
                    add_list.append(df[col].values[i])
                elif col in add_dict.keys():
                    add_list.append(add_dict[col])
                elif col == 'CIIE':
                    add_list.append(False)
                else:
                    print(f'{col} was not found')
            agg_summary_list.append(add_list)
    return pd.DataFrame(columns=agg_summary_columns,data=agg_summary_list)

In [5]:
import glob
fnames = glob.glob('./naive/*.pkl')
df_naive_agg = naive_pkl_aggregation(fnames)

In [6]:
fnames = glob.glob('./diff/*.pkl')
df_diff_agg = diff_pkl_aggregation(fnames)

In [7]:
fnames = glob.glob('./term_diff/*.pkl')
df_diff_split_select_agg = diff_pkl_aggregation(fnames)

In [8]:
# make naive dataframe with average/std of replicate simulations
naive_groupby_cols = ['circuit', 'selection', 'muP_single_cassette','n_cassettes','burden', 'Kpop', 
                      'kMB', 'kPL', 'Vmax','Km', 'MIC', 'abx', 'V']
df_naive_agg_avg = df_naive_agg.groupby(naive_groupby_cols,as_index=False)['total_production'].mean()
df_naive_agg_avg['std'] = df_naive_agg.groupby(naive_groupby_cols)['total_production'].std().values

In [9]:
# make diff dataframe with average/std of replicate simulations
diff_groupby_cols = ['circuit','n_cassettes','n_int_cassettes','selection','CIIE',
                     'n_div','burden', 'Kpop','kdiff', 'kMB', 'kMD','kMI', 'kPL', 'Vmax','Km', 'MIC', 'abx', 'V']
df_diff_agg_avg = df_diff_agg.groupby(diff_groupby_cols,as_index=False)['total_production'].mean()
df_diff_agg_avg['std'] = df_diff_agg.groupby(diff_groupby_cols)['total_production'].std().values

In [10]:
# make diff_split_select dataframe with average/std of replicate simulations
diff_split_select_groupby_cols = ['circuit','n_cassettes','n_int_cassettes','selection','CIIE',
                     'n_div','burden', 'Kpop','kdiff', 'kMB', 'kMD','kMI', 'kPL', 'Vmax','Km', 'MIC', 'abx', 'V']
df_diff_split_select_agg_avg = df_diff_split_select_agg.groupby(diff_split_select_groupby_cols,as_index=False)['total_production'].mean()
df_diff_split_select_agg_avg['std'] = df_diff_split_select_agg.groupby(diff_split_select_groupby_cols)['total_production'].std().values

In [11]:
# make K normalized df
df_naive_agg_avg['tot_prod_Knorm'] = df_naive_agg_avg['total_production'].values/df_naive_agg_avg['Kpop'].values
df_naive_agg_avg['std_Knorm'] = df_naive_agg_avg['std'].values/df_naive_agg_avg['Kpop'].values
# make K normalized df
df_diff_agg_avg['tot_prod_Knorm'] = df_diff_agg_avg['total_production'].values/df_diff_agg_avg['Kpop'].values
df_diff_agg_avg['std_Knorm'] = df_diff_agg_avg['std'].values/df_diff_agg_avg['Kpop'].values
# make K normalized df
df_diff_split_select_agg_avg['tot_prod_Knorm'] = df_diff_split_select_agg_avg['total_production'].values/df_diff_split_select_agg_avg['Kpop'].values
df_diff_split_select_agg_avg['std_Knorm'] = df_diff_split_select_agg_avg['std'].values/df_diff_split_select_agg_avg['Kpop'].values

In [13]:
#save pkl files
df_naive_agg.to_pickle('df_naive_agg.pkl')
df_diff_agg.to_pickle('df_diff_agg.pkl')
df_diff_split_select_agg.to_pickle('df_diff_split_select_agg.pkl')
df_diff_agg_avg.to_pickle('df_naive_agg_avg.pkl')
df_diff_split_select_agg_avg.to_pickle('df_diff_agg_avg.pkl')
df_diff_split_select_agg_avg.to_pickle('df_diff_split_select_agg_avg.pkl')

In [14]:
#load pkl files
df_naive_agg = pd.read_pickle('df_naive_agg.pkl')
df_diff_agg = pd.read_pickle('df_diff_agg.pkl')
df_diff_split_select_agg = pd.read_pickle('df_diff_split_select_agg.pkl')
df_diff_agg_avg = pd.read_pickle('df_naive_agg_avg.pkl')
df_diff_split_select_agg_avg = pd.read_pickle('df_diff_agg_avg.pkl')
df_diff_split_select_agg_avg = pd.read_pickle('df_diff_split_select_agg_avg.pkl')

Below we generate the plots shown in Figure 3C-E characterizing the impact of plasmid mutation and antibiotic degradation/plasmid loss on 1x and 2x naive, differentiation and terminal diifferentiation circuits.

In [ ]:
# first add variant column to df for ease of plotting. Will be in the order of no abx deg, 1x deg, 5x deg, 
# then 1e-8 plasmid mutation, 1e-6, and 1e-4 x μ
vmax_vals = np.sort(df_diff_agg.Vmax.unique())
plotting_dict = {'noabxdeg':{'Vmax':vmax_vals[0],
                                   'abx':100,
                                   'kPL':1e-4},
                 'abxdeg_1x':{'Vmax':vmax_vals[1],
                           'abx':100,
                           'kPL':1e-4},
                 'abxdeg_5x':{'Vmax':vmax_vals[2],
                           'abx':100,
                           'kPL':1e-4},
                 'plasmid_mutation_1e8':{'Vmax':vmax_vals[0],
                           'abx':0,
                           'kPL':1e-8},
                 'plasmid_mutation_1e6':{'Vmax':vmax_vals[0],
                           'abx':0,
                           'kPL':1e-6},
                 'plasmid_mutation_1e4':{'Vmax':vmax_vals[0],
                           'abx':0,
                           'kPL':1e-4}}
df_naive_agg_avg['variant'] = 0
df_diff_agg_avg['variant'] = 0
df_diff_split_select_agg_avg['variant'] = 0
for i, variant in enumerate(list(plotting_dict.keys())):
    Vmax = plotting_dict[variant]['Vmax']
    abx = plotting_dict[variant]['abx']
    kPL = plotting_dict[variant]['kPL'] 
    df_naive_agg_avg.loc[(df_naive_agg_avg.Vmax==Vmax)&\
                          (df_naive_agg_avg.abx==abx)&\
                          (df_naive_agg_avg.kPL==kPL),'variant'] = i+1
    df_diff_agg_avg.loc[(df_diff_agg_avg.Vmax==Vmax)&\
                          (df_diff_agg_avg.abx==abx)&\
                          (df_diff_agg_avg.kPL==kPL),'variant'] = i+1
    df_diff_split_select_agg_avg.loc[(df_diff_split_select_agg_avg.Vmax==Vmax)&\
                          (df_diff_split_select_agg_avg.abx==abx)&\
                          (df_diff_split_select_agg_avg.kPL==kPL),'variant'] = i+1
    df_naive_agg.loc[(df_naive_agg.Vmax==Vmax)&\
                          (df_naive_agg.abx==abx)&\
                          (df_naive_agg.kPL==kPL),'variant'] = i+1
    df_diff_agg.loc[(df_diff_agg.Vmax==Vmax)&\
                          (df_diff_agg.abx==abx)&\
                          (df_diff_agg.kPL==kPL),'variant'] = i+1
    df_diff_split_select_agg.loc[(df_diff_split_select_agg.Vmax==Vmax)&\
                          (df_diff_split_select_agg.abx==abx)&\
                          (df_diff_split_select_agg.kPL==kPL),'variant'] = i+1

In [30]:
# make plot with only 10%, 50%, 90% burden (top to bottom) as shown in figure 3C-E
burdens = [10,50,90]
kdiffs = df_diff_agg_avg.kdiff.unique()
selection = 'additive'
n_cassettes_array = [1,2]
marker_size = 4
ndiv=4
alpha_array = np.linspace(0.2,0.8,8)
offsets = np.linspace(-0.25,0.25,8)
naive_offsets = np.linspace(-0.2,0.2,3)
capsize=2
sns.set_palette('colorblind',8)
naive_colors = [(150/255,150/255,150/255),(37/255,37/255,37/255)]
fig, ax = plt.subplots(3,3,sharey=True,sharex=True,figsize=(12,15))

for i, burden in enumerate(burdens):
    ax[i,0].set_title(f'1x diff: {burden}% burden',fontsize=14)
    ax[i,1].set_title(f'2x diff: {burden}% burden',fontsize=14) 
    ax[i,2].set_title(f'naive: {burden}% burden',fontsize=14)
    ax[i,0].set_ylabel('production/K',fontsize=14)
    for k, kdiff in enumerate(kdiffs):
        df = df_diff_agg_avg.loc[(df_diff_agg_avg.burden==burden)&\
                          (df_diff_agg_avg.kdiff==kdiff),:]
        for j, n_cassettes in enumerate(n_cassettes_array):
            ax[i,j].errorbar(df.loc[df.n_cassettes==n_cassettes,'variant'].values + offsets[k],
                             df.loc[df.n_cassettes==n_cassettes,'tot_prod_Knorm'].values,
                             yerr=df.loc[df.n_cassettes==n_cassettes,'std_Knorm'].values,
                             color=sns.color_palette()[0],marker='o',linestyle='None',ms=marker_size,
                             alpha=alpha_array[k],capsize=capsize
                                )
            ax[i,j].set_xticks([1,2,3,4,5,6])
        df = df_diff_split_select_agg_avg.loc[(df_diff_split_select_agg_avg.burden==burden)&\
                          (df_diff_split_select_agg_avg.kdiff==kdiff)&\
                          (df_diff_split_select_agg_avg.n_div==ndiv),:]
        for j, n_cassettes in enumerate(n_cassettes_array):
            ax[i,j].errorbar(df.loc[df.n_cassettes==n_cassettes,'variant'].values+offsets[k],
                             df.loc[df.n_cassettes==n_cassettes,'tot_prod_Knorm'].values,
                             yerr=df.loc[df.n_cassettes==n_cassettes,'std_Knorm'].values,
                             color=sns.color_palette()[1],marker='o',linestyle='None',ms=marker_size,
                             alpha=alpha_array[k],capsize=capsize
                                )
            ax[i,j].set_xticks([1,2,3,4,5,6])
            
    df = df_naive_agg_avg.loc[(df_naive_agg_avg.burden==burden)&\
                      (df_naive_agg_avg.selection==selection),:]
    for j, n_cassettes in enumerate(n_cassettes_array):
        ax[i,2].errorbar(df.loc[(df.n_cassettes==n_cassettes)&\
                                (df.muP_single_cassette==False),'variant'].values + naive_offsets[j],
                         df.loc[(df.n_cassettes==n_cassettes)&\
                                (df.muP_single_cassette==False),'tot_prod_Knorm'].values,
                         yerr=df.loc[(df.n_cassettes==n_cassettes)&\
                                (df.muP_single_cassette==False),'std_Knorm'].values,
                         color=naive_colors[j],marker='o',linestyle='None',ms=marker_size,
                         alpha=0.6,capsize=capsize
                            )
    ax[i,2].errorbar(df.loc[(df.n_cassettes==2)&\
                                (df.muP_single_cassette==True),'variant'].values+ naive_offsets[2],
                         df.loc[(df.n_cassettes==2)&\
                                (df.muP_single_cassette==True),'tot_prod_Knorm'].values,
                         yerr=df.loc[(df.n_cassettes==2)&\
                                (df.muP_single_cassette==True),'std_Knorm'].values,
                         color=naive_colors[1],marker='s',linestyle='None',ms=marker_size,
                         alpha=0.6,capsize=capsize
                            )
    ax[i,2].set_xticks([1,2,3,4,5,6])
ax[0,0].set_yscale('log')
ax[0,0].legend(['diff','term. diff'],fontsize=14,loc='best')
ax[0,1].legend(['diff','term. diff'],fontsize=14,loc='best')
ax[0,2].legend(['1x','2x','2x*'],loc='best', fontsize=14)

plt.savefig('./20220810_allvariants_all_strains_105090_burden_logscale.pdf')

1 extra bytes in post.stringData array
'created' timestamp seems very low; regarding as unix timestamp
Zapf NOT subset; don't know how to subset; dropped
feat NOT subset; don't know how to subset; dropped
meta NOT subset; don't know how to subset; dropped
morx NOT subset; don't know how to subset; dropped


Below we generate the plots shown in Supplementary Figure 17 examining the impact of number of post-differentiation cell divisions in 1x/2x terminal differentiation on total production in the context of plasmid mutation and plasmid loss/communal antibiotic degradation.

In [19]:
#2 vs 4 vs 8 div 1/17/22
burdens = [10,50,90]
kdiffs = df_diff_agg_avg.kdiff.unique()
variant = 'abxdeg'
selection = 'additive'
n_cassettes_array = [1,2]
marker_size = 4
alpha_array = np.linspace(0.2,0.8,8)
offsets = np.linspace(-0.25,0.25,8)
capsize=2
sns.set_palette('colorblind',8)
fig, ax = plt.subplots(3,3,sharey=True,sharex=True,figsize=(12,15)) 

for i, burden in enumerate(burdens):
    ax[i,0].set_ylabel('production/K',fontsize=18)
    for n, ndiv in enumerate([2,4,8]):
        for k, kdiff in enumerate(kdiffs):
            df = df_diff_split_select_agg_avg.loc[(df_diff_split_select_agg_avg.burden==burden)&\
                              (df_diff_split_select_agg_avg.kdiff==kdiff)&\
                              (df_diff_split_select_agg_avg.n_div==ndiv),:]
            for j, n_cassettes in enumerate(n_cassettes_array):
                ax[i,n].errorbar(df.loc[df.n_cassettes==n_cassettes,'variant'].values+offsets[k],
                                 df.loc[df.n_cassettes==n_cassettes,'tot_prod_Knorm'].values,
                                 yerr=df.loc[df.n_cassettes==n_cassettes,'std_Knorm'].values,
                                 color=sns.color_palette()[j],marker='o',linestyle='None',ms=marker_size,
                                 alpha=alpha_array[k],capsize=capsize
                                    )
                ax[i,n].set_xticks([1,2,3,4,5,6])
ax[0,0].set_yscale('log')
ax[0,0].legend(['1x term. diff','2x term. diff'],loc='best', fontsize=14)
ax[0,0].set_title('2 divisions',fontsize=18)
ax[0,1].set_title('4 divisions',fontsize=18)
ax[0,2].set_title('8 divisions',fontsize=18)   
# ax[2,0].legend(['1x','2x','3x','4x'],loc='best',fontsize=14)
plt.savefig('./20220117_allvariants_termdiff_248_ndiv_logscale.pdf')

Text(0.5, 1.0, '8 divisions')

Below we will generate timecourse plots shown in Supplementary Figures 18-20 showing tiime-course production, producer/non-producer populations, and degrader/cheater (WT plasmid/mutant plasmid) populations, withh and without plasmid effects forr naive 1x and 2x. We will only plot 3 of the 8 stochastic simulations.

In [24]:
# naive plot with markers
"""
row 1: production
row 2: producers/non-producers
row 3: degraders/cheaters
"""
circuit = 'naive'
n_cassette_array = np.array([1,2])
# n_cassette_array = np.array([1])
muN=2
reps = [1,2,3]
muPs = [0.1,0.5,0.9]*muN
variants = [1,2,3,4,5,6]
df = df_naive_agg
lw=1
ms=3
burdens = [10,50,90]
titles = ['no plasmid effects','1x abx deg','5x abx deg',f'10^{-8} plasm mut',f'10^{-6} plasm mut',f'10^{-4} plasm mut']
for burden in burdens:
    fig, ax = plt.subplots(3,6,sharey='row',sharex=True,figsize=(32,15))  
    for i, variant in enumerate(variants):
        ax[0,i].set_title(titles[i],fontsize=14)
        ax[2,i].set_xlabel('plate generation',fontsize=14)
        for k, rep in enumerate(reps):
            for j, n_cassettes in enumerate(n_cassette_array):
                pop_tots = df.loc[(df.n_cassettes==n_cassettes)&\
                            (df.burden==burden)&\
                            (df.rep==rep)&\
                            (df.variant==variant)&\
                                (df.muP_single_cassette==False),'pop_tot'].values[0]
                producer_tots = df.loc[(df.n_cassettes==n_cassettes)&\
                            (df.burden==burden)&\
                            (df.rep==rep)&\
                            (df.variant==variant)&\
                                (df.muP_single_cassette==False),'producers'].values[0]
                nonproducer_tots = pop_tots - producer_tots
                degrader_tots = df.loc[(df.n_cassettes==n_cassettes)&\
                            (df.burden==burden)&\
                            (df.rep==rep)&\
                            (df.variant==variant)&\
                                (df.muP_single_cassette==False),'degraders'].values[0]
                cheater_tots = df.loc[(df.n_cassettes==n_cassettes)&\
                            (df.burden==burden)&\
                            (df.rep==rep)&\
                            (df.variant==variant)&\
                                (df.muP_single_cassette==False),'cheaters'].values[0]
                prod = df.loc[(df.n_cassettes==n_cassettes)&\
                              (df.burden==burden)&\
                              (df.rep==rep)&\
                            (df.variant==variant)&\
                                (df.muP_single_cassette==False),'production_array'].values[0]/1e10
                ax[0,i].plot(np.arange(1,21),prod,'-',color=sns.color_palette()[j],alpha=0.3,lw=lw,marker='o',ms=ms)
                ax[1,i].plot(np.arange(1,21),producer_tots/1e9,'-',color=sns.color_palette()[j],alpha=0.8,lw=lw,marker='o',ms=ms)
                ax[1,i].plot(np.arange(1,21),nonproducer_tots/1e9,'--',color=sns.color_palette()[j],alpha=0.8,lw=lw,marker='o',mfc='white',ms=ms)
                ax[2,i].plot(np.arange(1,21),degrader_tots/1e9,'-',color=sns.color_palette()[j],alpha=0.8,lw=lw,marker='o',ms=ms)
                ax[2,i].plot(np.arange(1,21),cheater_tots/1e9,'--',color=sns.color_palette()[j],alpha=0.8,lw=lw,marker='o',mfc='white',ms=ms) 
                if n_cassettes == 2:
                    pop_tots = df.loc[(df.n_cassettes==n_cassettes)&\
                            (df.burden==burden)&\
                            (df.rep==rep)&\
                                      (df.variant==variant)&\
                            (df.muP_single_cassette==True),'pop_tot'].values[0]
                    producer_tots = df.loc[(df.n_cassettes==n_cassettes)&\
                                (df.burden==burden)&\
                                (df.rep==rep)&\
                                           (df.variant==variant)&\
                                (df.muP_single_cassette==True),'producers'].values[0]
                    nonproducer_tots = df.loc[(df.n_cassettes==n_cassettes)&\
                                (df.burden==burden)&\
                                (df.rep==rep)&\
                                              (df.variant==variant)&\
                                (df.muP_single_cassette==True),'mutants'].values[0]
                    degrader_tots = df.loc[(df.n_cassettes==n_cassettes)&\
                                (df.burden==burden)&\
                                (df.rep==rep)&\
                                           (df.variant==variant)&\
                                (df.muP_single_cassette==True),'degraders'].values[0]
                    cheater_tots = df.loc[(df.n_cassettes==n_cassettes)&\
                                (df.burden==burden)&\
                                (df.rep==rep)&\
                                          (df.variant==variant)&\
                                (df.muP_single_cassette==True),'cheaters'].values[0]
                    prod = df.loc[(df.n_cassettes==n_cassettes)&\
                                (df.burden==burden)&\
                                  (df.rep==rep)&\
                                  (df.variant==variant)&\
                                (df.muP_single_cassette==True),'production_array'].values[0]/1e10
                    ax[0,i].plot(np.arange(1,21),prod,'-',color=sns.color_palette()[2],alpha=0.3,marker='o',ms=ms)
                    ax[1,i].plot(np.arange(1,21),producer_tots/1e9,'-',color=sns.color_palette()[2],alpha=0.8,lw=lw,marker='o',ms=ms)
                    ax[1,i].plot(np.arange(1,21),nonproducer_tots/1e9,'--',color=sns.color_palette()[2],alpha=0.8,lw=lw,marker='o',mfc='white',ms=ms)
                    ax[2,i].plot(np.arange(1,21),degrader_tots/1e9,'-',color=sns.color_palette()[2],alpha=0.8,lw=lw,marker='o',ms=ms)
                    ax[2,i].plot(np.arange(1,21),cheater_tots/1e9,'--',color=sns.color_palette()[2],alpha=0.8,lw=lw,marker='o',mfc='white',ms=ms)
 
        ax[0,0].set_ylabel('prod. (x$10^{10}$ a.u.)',fontsize=14)
        ax[1,0].set_ylabel('cells (x$10^9$)',fontsize=14)
        ax[2,0].set_ylabel('cells (x$10^9$)',fontsize=14)
        ax[0,0].legend(['1x','2x','2x*'])
        ax[1,0].legend(['producers','nonproducers'],fontsize=12)
        ax[2,0].legend(['degraders','cheaters'],fontsize=12)
        ax[1,3].legend(['producers','nonproducers'],fontsize=12)    
        ax[2,3].legend(['WT plas.','mut plas.'],fontsize=12)
    plt.savefig(f'./20220119_naive_allvariants_timecourse_{burden}burden.pdf')

1 extra bytes in post.stringData array
'created' timestamp seems very low; regarding as unix timestamp
Zapf NOT subset; don't know how to subset; dropped
feat NOT subset; don't know how to subset; dropped
meta NOT subset; don't know how to subset; dropped
morx NOT subset; don't know how to subset; dropped
1 extra bytes in post.stringData array
'created' timestamp seems very low; regarding as unix timestamp
Zapf NOT subset; don't know how to subset; dropped
feat NOT subset; don't know how to subset; dropped
meta NOT subset; don't know how to subset; dropped
morx NOT subset; don't know how to subset; dropped
1 extra bytes in post.stringData array
'created' timestamp seems very low; regarding as unix timestamp
Zapf NOT subset; don't know how to subset; dropped
feat NOT subset; don't know how to subset; dropped
meta NOT subset; don't know how to subset; dropped
morx NOT subset; don't know how to subset; dropped


Below we will generate timecourse plots shown in Supplementary Figures 21-23 (1x diff and term. diff.), and Figure Supplementary Figures 24-26 (2x diff and term. diff), showing time-course production, producer/non-producer populations, and degrader/cheater (WT plasmid/mutant plasmid) populations, with and without plasmid effects. We will only plot 3 of the 8 stochastic simulations.

In [29]:
# Loop through and generate 1x diff/term diff, and 2x diff/term diff plots for 10, 50, 90% percent burden.
"""
row 1: production
row 2: producers/non-producers
row 3: degraders/cheaters
"""
muN=2
n_div = 4
reps = [1,2,3]
variants = [1,2,3,4,5,6]
lw=1
ms=3
kdiff = 0.8
n_cassettes_array = [1,2]
titles = ['no plasmid effects','1x abx deg','5x abx deg',f'10^{-8} plasm mut',f'10^{-6} plasm mut',f'10^{-4} plasm mut']
burdens = [10,50,90]
for n_cassettes in n_cassettes_array:
    for burden in burdens:
        fig, ax = plt.subplots(3,6,sharey='row',sharex=True,figsize=(32,15)) 
        for i, variant in enumerate(variants):
            ax[0,i].set_title(titles[i],fontsize=14)
            ax[2,i].set_xlabel('plate generation',fontsize=14)
            for k, rep in enumerate(reps):
                df = df_diff_agg
                j=0
                pop_tots = df.loc[(df.n_cassettes==n_cassettes)&\
                                  (df.burden==burden)&\
                                  (df.rep==rep)&\
                                  (df.variant==variant)&\
                                  (df.kdiff==kdiff),'pop_tot'].values[0]
                producer_tots = df.loc[(df.n_cassettes==n_cassettes)&\
                                  (df.burden==burden)&\
                                  (df.rep==rep)&\
                                  (df.variant==variant)&\
                                  (df.kdiff==kdiff),'producers'].values[0]
                nonproducer_tots = pop_tots - producer_tots
                degrader_tots = df.loc[(df.n_cassettes==n_cassettes)&\
                                  (df.burden==burden)&\
                                  (df.rep==rep)&\
                                  (df.variant==variant)&\
                                  (df.kdiff==kdiff),'degraders'].values[0]
                cheater_tots = df.loc[(df.n_cassettes==n_cassettes)&\
                                  (df.burden==burden)&\
                                  (df.rep==rep)&\
                                  (df.variant==variant)&\
                                  (df.kdiff==kdiff),'cheaters'].values[0]
                prod = df.loc[(df.n_cassettes==n_cassettes)&\
                                  (df.burden==burden)&\
                                  (df.rep==rep)&\
                                  (df.variant==variant)&\
                                  (df.kdiff==kdiff),'production_array'].values[0]/1e10
                ax[0,i].plot(np.arange(1,21),prod,'-',color=sns.color_palette()[j],alpha=0.3,lw=lw,marker='o',ms=ms)
                ax[1,i].plot(np.arange(1,21),producer_tots/1e9,'-',color=sns.color_palette()[j],alpha=0.8,lw=lw,marker='o',ms=ms)
                ax[1,i].plot(np.arange(1,21),nonproducer_tots/1e9,'--',color=sns.color_palette()[j],alpha=0.8,lw=lw,marker='o',mfc='white',ms=ms)
                ax[2,i].plot(np.arange(1,21),degrader_tots/1e9,'-',color=sns.color_palette()[j],alpha=0.8,lw=lw,marker='o',ms=ms)
                ax[2,i].plot(np.arange(1,21),cheater_tots/1e9,'--',color=sns.color_palette()[j],alpha=0.8,lw=lw,marker='o',mfc='white',ms=ms) 
                df = df_diff_split_select_agg
                j=1
                pop_tots = df.loc[(df.n_cassettes==n_cassettes)&\
                                  (df.burden==burden)&\
                                  (df.rep==rep)&\
                                  (df.variant==variant)&\
                                  (df.kdiff==kdiff)&\
                                  (df.n_div==n_div),'pop_tot'].values[0]
                producer_tots = df.loc[(df.n_cassettes==n_cassettes)&\
                                  (df.burden==burden)&\
                                  (df.rep==rep)&\
                                  (df.variant==variant)&\
                                  (df.kdiff==kdiff)&\
                                  (df.n_div==n_div),'producers'].values[0]
                nonproducer_tots = pop_tots - producer_tots
                degrader_tots = df.loc[(df.n_cassettes==n_cassettes)&\
                                  (df.burden==burden)&\
                                  (df.rep==rep)&\
                                  (df.variant==variant)&\
                                  (df.kdiff==kdiff)&\
                                  (df.n_div==n_div),'degraders'].values[0]
                cheater_tots = df.loc[(df.n_cassettes==n_cassettes)&\
                                  (df.burden==burden)&\
                                  (df.rep==rep)&\
                                  (df.variant==variant)&\
                                  (df.kdiff==kdiff)&\
                                  (df.n_div==n_div),'cheaters'].values[0]
                prod = df.loc[(df.n_cassettes==n_cassettes)&\
                                  (df.burden==burden)&\
                                  (df.rep==rep)&\
                                  (df.variant==variant)&\
                                  (df.kdiff==kdiff)&\
                                  (df.n_div==n_div),'production_array'].values[0]/1e10
                ax[0,i].plot(np.arange(1,21),prod,'-',color=sns.color_palette()[j],alpha=0.3,lw=lw,marker='o',ms=ms)
                ax[1,i].plot(np.arange(1,21),producer_tots/1e9,'-',color=sns.color_palette()[j],alpha=0.8,lw=lw,marker='o',ms=ms)
                ax[1,i].plot(np.arange(1,21),nonproducer_tots/1e9,'--',color=sns.color_palette()[j],alpha=0.8,lw=lw,marker='o',mfc='white',ms=ms)
                ax[2,i].plot(np.arange(1,21),degrader_tots/1e9,'-',color=sns.color_palette()[j],alpha=0.8,lw=lw,marker='o',ms=ms)
                ax[2,i].plot(np.arange(1,21),cheater_tots/1e9,'--',color=sns.color_palette()[j],alpha=0.8,lw=lw,marker='o',mfc='white',ms=ms) 

            ax[0,0].legend(['diff','term. diff.'])
            ax[0,0].set_ylabel('prod. (x$10^{10}$ a.u.)',fontsize=14)
            ax[1,0].set_ylabel('cells (x$10^9$)',fontsize=14)
            ax[2,0].set_ylabel('cells (x$10^9$)',fontsize=14)
            ax[1,0].legend(['producers','nonproducers'],fontsize=12)
            ax[2,0].legend(['degraders','cheaters'],fontsize=12)
            ax[1,3].legend(['producers','nonproducers'],fontsize=12)    
            ax[2,3].legend(['WT plas.','mut plas.'],fontsize=12)

        plt.savefig(f'./20220119_diff_termdiff_{n_cassettes}x_allvariants_timecourse_{burden}burden_80kdiff.pdf')

1 extra bytes in post.stringData array
'created' timestamp seems very low; regarding as unix timestamp
Zapf NOT subset; don't know how to subset; dropped
feat NOT subset; don't know how to subset; dropped
meta NOT subset; don't know how to subset; dropped
morx NOT subset; don't know how to subset; dropped
1 extra bytes in post.stringData array
'created' timestamp seems very low; regarding as unix timestamp
Zapf NOT subset; don't know how to subset; dropped
feat NOT subset; don't know how to subset; dropped
meta NOT subset; don't know how to subset; dropped
morx NOT subset; don't know how to subset; dropped
1 extra bytes in post.stringData array
'created' timestamp seems very low; regarding as unix timestamp
Zapf NOT subset; don't know how to subset; dropped
feat NOT subset; don't know how to subset; dropped
meta NOT subset; don't know how to subset; dropped
morx NOT subset; don't know how to subset; dropped
1 extra bytes in post.stringData array
'created' timestamp seems very low; rega